In [1]:
import pandas as pd

In [1]:
# import os
# os.environ['OPENAI_API_KEY'] = 'key'

In [4]:
from openai import OpenAI

client = OpenAI()

In [26]:
df = pd.read_csv('../data/data.csv')
df.columns = df.columns.str.lower().str.replace(' ', '_')
# df.insert(0, 'id', df.index)
documents = df.to_dict(orient='records')

In [8]:
documents[0]

{'exercise_name': 'Push-Ups',
 'type_of_activity': 'Strength',
 'type_of_equipment': 'Bodyweight',
 'body_part': 'Upper Body',
 'type': 'Push',
 'muscle_groups_activated': 'Pectorals, Triceps, Deltoids',
 'instructions': 'Start in a high plank position with your hands under your shoulders. Lower your body until your chest nearly touches the floor. Push back up to the starting position.'}

In [27]:
df

,id,exercise_name,type_of_activity,type_of_equipment,body_part,type,muscle_groups_activated,instructions
0,0,Push-Ups,Strength,Bodyweight,Upper Body,Push,"Pectorals, Triceps, Deltoids",Start in a high plank position with your hands...
1,1,Squats,Strength,Bodyweight,Lower Body,Push,"Quadriceps, Glutes, Hamstrings",Stand with feet shoulder-width apart. Lower yo...
2,2,Plank,Strength/Mobility,Bodyweight,Core,Hold,"Rectus Abdominis, Transverse Abdominis",Start in a forearm plank position with your el...
3,3,Deadlift,Strength,Barbell,Lower Body,Pull,"Glutes, Hamstrings, Lower Back","Stand with feet hip-width apart, barbell in fr..."
4,4,Bicep Curls,Strength,Dumbbells,Upper Body,Pull,"Biceps, Forearms","Stand with a dumbbell in each hand, arms fully..."
...,...,...,...,...,...,...,...,...
202,202,Incline Dumbbell Row,Strength,Dumbbells,Upper Body,Pull,"Latissimus Dorsi, Biceps",Lie face down on an incline bench with a dumbb...
203,203,Machine Lat Pulldown,Strength,Machine,Upper Body,Pull,"Latissimus Dorsi, Biceps",Sit at a lat pulldown machine with a wide grip...
204,204,One-Arm Cable Row,Strength,Cable Machine,Upper Body,Pull,"Latissimus Dorsi, Biceps",Stand facing a cable machine with the handle a...
205,205,Face Pull,Strength,Cable Machine,Upper Body,Pull,"Rear Deltoids, Trapezius, Rhomboids",Attach a rope handle to a high cable pulley. P...


In [28]:
prompt_template = """
You emulate a user of our fitness assistant application.
Formulate 5 questions this user might ask based on a provided exercise.
Make the questions specific to this exercise.
The record should contain the answer to the questions, and the questions should
be complete and not too short. Use as fewer words as possible from the record. 

The record:

exercise_name: {exercise_name}
type_of_activity: {type_of_activity}
type_of_equipment: {type_of_equipment}
body_part: {body_part}
type: {type}
muscle_groups_activated: {muscle_groups_activated}
instructions: {instructions}

Provide the output in parsable JSON without using code blocks:

{{"questions": ["question1", "question2", ..., "question5"]}}
""".strip()

In [29]:
prompt = prompt_template.format(**documents[0])

In [30]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [31]:
questions = llm(prompt)

In [32]:
import json

In [33]:
json.loads(questions)

{'questions': ['What is the primary body position to start performing push-ups?',
  'Which muscle groups are primarily activated during push-ups?',
  'What movement do you perform after lowering your body in a push-up?',
  'Is any equipment needed to perform push-ups?',
  'What is the starting position for executing a push-up correctly?']}

In [34]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [35]:
from tqdm.auto import tqdm

In [36]:
results = {}

In [37]:
for doc in tqdm(documents):
    print(doc)
    doc_id = doc['id']
    if not doc_id:
        print(f"No 'id' found in document: {doc}")
    if doc_id in results:
        continue

    questions_raw = generate_questions(doc)
    questions = json.loads(questions_raw)
    results[doc_id] = questions['questions']

  0%|          | 0/207 [00:00<?, ?it/s]

{'id': 0, 'exercise_name': 'Push-Ups', 'type_of_activity': 'Strength', 'type_of_equipment': 'Bodyweight', 'body_part': 'Upper Body', 'type': 'Push', 'muscle_groups_activated': 'Pectorals, Triceps, Deltoids', 'instructions': 'Start in a high plank position with your hands under your shoulders. Lower your body until your chest nearly touches the floor. Push back up to the starting position.'}
No 'id' found in document: {'id': 0, 'exercise_name': 'Push-Ups', 'type_of_activity': 'Strength', 'type_of_equipment': 'Bodyweight', 'body_part': 'Upper Body', 'type': 'Push', 'muscle_groups_activated': 'Pectorals, Triceps, Deltoids', 'instructions': 'Start in a high plank position with your hands under your shoulders. Lower your body until your chest nearly touches the floor. Push back up to the starting position.'}
{'id': 1, 'exercise_name': 'Squats', 'type_of_activity': 'Strength', 'type_of_equipment': 'Bodyweight', 'body_part': 'Lower Body', 'type': 'Push', 'muscle_groups_activated': 'Quadriceps

{'id': 22, 'exercise_name': 'Side Plank', 'type_of_activity': 'Strength', 'type_of_equipment': 'Bodyweight', 'body_part': 'Core', 'type': 'Hold', 'muscle_groups_activated': 'Obliques, Shoulders', 'instructions': 'Lie on one side with your legs straight. Lift your hips so your body forms a straight line, supporting yourself on one forearm.'}
{'id': 23, 'exercise_name': 'Cable Flyes', 'type_of_activity': 'Strength', 'type_of_equipment': 'Cable Machine', 'body_part': 'Upper Body', 'type': 'Push', 'muscle_groups_activated': 'Pectorals', 'instructions': 'Stand in the middle of a cable machine with handles in each hand. With a slight bend in your elbows, bring your hands together in front of your chest.'}
{'id': 24, 'exercise_name': 'Step-Ups', 'type_of_activity': 'Strength', 'type_of_equipment': 'Bodyweight', 'body_part': 'Lower Body', 'type': 'Push', 'muscle_groups_activated': 'Quadriceps, Glutes', 'instructions': 'Step onto a bench or box with one foot, then push through that foot to lift

{'id': 46, 'exercise_name': 'Bulgarian Split Squats', 'type_of_activity': 'Strength', 'type_of_equipment': 'Bodyweight', 'body_part': 'Lower Body', 'type': 'Push', 'muscle_groups_activated': 'Quadriceps, Glutes, Hamstrings', 'instructions': 'Stand lunge-style with one foot behind you on a bench. Lower your hips until your front thigh is parallel to the floor, then push back up.'}
{'id': 47, 'exercise_name': 'Seated Shoulder Press', 'type_of_activity': 'Strength', 'type_of_equipment': 'Dumbbells', 'body_part': 'Upper Body', 'type': 'Push', 'muscle_groups_activated': 'Deltoids, Triceps', 'instructions': 'Sit on a bench with a dumbbell in each hand at shoulder height. Press the weights overhead until your arms are fully extended.'}
{'id': 48, 'exercise_name': 'Incline Push-Ups', 'type_of_activity': 'Strength', 'type_of_equipment': 'Bodyweight', 'body_part': 'Upper Body', 'type': 'Push', 'muscle_groups_activated': 'Pectorals, Triceps, Deltoids', 'instructions': 'Place your hands on an elev

{'id': 70, 'exercise_name': 'Tuck Jumps', 'type_of_activity': 'Cardio/Strength', 'type_of_equipment': 'Bodyweight', 'body_part': 'Lower Body', 'type': 'Push', 'muscle_groups_activated': 'Quadriceps, Glutes, Calves', 'instructions': 'Jump straight up and tuck your knees towards your chest, then land softly and repeat.'}
{'id': 71, 'exercise_name': 'Windshield Wipers', 'type_of_activity': 'Strength', 'type_of_equipment': 'Bodyweight', 'body_part': 'Core', 'type': 'Hold', 'muscle_groups_activated': 'Obliques, Rectus Abdominis', 'instructions': 'Lie on your back with legs extended towards the ceiling. Rotate your legs side to side like windshield wipers.'}
{'id': 72, 'exercise_name': 'Dumbbell Shrugs', 'type_of_activity': 'Strength', 'type_of_equipment': 'Dumbbells', 'body_part': 'Upper Body', 'type': 'Hold', 'muscle_groups_activated': 'Trapezius', 'instructions': 'Stand with a dumbbell in each hand at your sides. Shrug your shoulders towards your ears, then lower them back down.'}
{'id': 

{'id': 95, 'exercise_name': 'Machine Leg Curl', 'type_of_activity': 'Strength', 'type_of_equipment': 'Machine', 'body_part': 'Lower Body', 'type': 'Pull', 'muscle_groups_activated': 'Hamstrings', 'instructions': 'Sit on a leg curl machine and curl the pad down towards your glutes, then return to the starting position.'}
{'id': 96, 'exercise_name': 'Incline Bench Press', 'type_of_activity': 'Strength', 'type_of_equipment': 'Barbell', 'body_part': 'Upper Body', 'type': 'Push', 'muscle_groups_activated': 'Pectorals, Triceps, Deltoids', 'instructions': 'Lie on an incline bench with a barbell. Press the bar up and together over your chest, then lower it back down.'}
{'id': 97, 'exercise_name': 'Resistance Band Deadlift', 'type_of_activity': 'Strength', 'type_of_equipment': 'Resistance Band', 'body_part': 'Lower Body', 'type': 'Pull', 'muscle_groups_activated': 'Glutes, Hamstrings, Lower Back', 'instructions': 'Stand on a resistance band with feet shoulder-width apart. Bend at the hips and k

{'id': 120, 'exercise_name': 'Machine Shoulder Press', 'type_of_activity': 'Strength', 'type_of_equipment': 'Machine', 'body_part': 'Upper Body', 'type': 'Push', 'muscle_groups_activated': 'Deltoids, Triceps', 'instructions': 'Sit on a shoulder press machine, grasp the handles, and press them overhead until your arms are fully extended.'}
{'id': 121, 'exercise_name': 'Sumo Squat', 'type_of_activity': 'Strength', 'type_of_equipment': 'Bodyweight', 'body_part': 'Lower Body', 'type': 'Push', 'muscle_groups_activated': 'Quadriceps, Glutes, Hamstrings', 'instructions': 'Stand with feet wider than shoulder-width apart, toes pointing out. Lower your hips until your thighs are parallel to the floor, then push back up.'}
{'id': 122, 'exercise_name': 'Resistance Band Squat', 'type_of_activity': 'Strength', 'type_of_equipment': 'Resistance Band', 'body_part': 'Lower Body', 'type': 'Push', 'muscle_groups_activated': 'Quadriceps, Glutes, Hamstrings', 'instructions': 'Stand on a resistance band with

{'id': 143, 'exercise_name': 'Zottman Curl', 'type_of_activity': 'Strength', 'type_of_equipment': 'Dumbbells', 'body_part': 'Upper Body', 'type': 'Pull', 'muscle_groups_activated': 'Biceps, Forearms', 'instructions': 'Stand with feet shoulder-width apart, holding a dumbbell in each hand. Curl the weights up with a supinated grip, then rotate your wrists and lower them with a pronated grip.'}
{'id': 144, 'exercise_name': 'Cable Crunch', 'type_of_activity': 'Strength', 'type_of_equipment': 'Cable Machine', 'body_part': 'Core', 'type': 'Hold', 'muscle_groups_activated': 'Rectus Abdominis', 'instructions': 'Kneel in front of a cable machine with a rope attachment. Crunch down, bringing your elbows towards your knees while keeping your back straight.'}
{'id': 145, 'exercise_name': 'Stability Ball Pike', 'type_of_activity': 'Strength', 'type_of_equipment': 'Stability Ball', 'body_part': 'Core', 'type': 'Hold', 'muscle_groups_activated': 'Rectus Abdominis, Hip Flexors', 'instructions': 'Start

{'id': 167, 'exercise_name': 'Step Mill', 'type_of_activity': 'Cardio', 'type_of_equipment': 'Machine', 'body_part': 'Lower Body', 'type': 'Push', 'muscle_groups_activated': 'Quadriceps, Glutes, Hamstrings, Calves', 'instructions': 'Step continuously on a revolving staircase, keeping a steady pace.'}
{'id': 168, 'exercise_name': 'Standing Resistance Band Row', 'type_of_activity': 'Strength', 'type_of_equipment': 'Resistance Band', 'body_part': 'Upper Body', 'type': 'Pull', 'muscle_groups_activated': 'Latissimus Dorsi, Biceps', 'instructions': 'Attach a resistance band to a sturdy anchor. Pull the handles towards your torso, keeping your elbows close to your body.'}
{'id': 169, 'exercise_name': 'Dumbbell Lunge', 'type_of_activity': 'Strength', 'type_of_equipment': 'Dumbbells', 'body_part': 'Lower Body', 'type': 'Push', 'muscle_groups_activated': 'Quadriceps, Glutes, Hamstrings', 'instructions': 'Hold a dumbbell in each hand and step forward into a lunge. Push back to the starting positi

{'id': 190, 'exercise_name': 'One-Arm Dumbbell Snatch', 'type_of_activity': 'Strength', 'type_of_equipment': 'Dumbbell', 'body_part': 'Full Body', 'type': 'Push', 'muscle_groups_activated': 'Glutes, Shoulders, Triceps', 'instructions': 'Start with a dumbbell between your feet. Explosively pull the dumbbell up and overhead in one motion.'}
{'id': 191, 'exercise_name': 'Cable Front Squat', 'type_of_activity': 'Strength', 'type_of_equipment': 'Cable Machine', 'body_part': 'Lower Body', 'type': 'Push', 'muscle_groups_activated': 'Quadriceps, Glutes, Hamstrings', 'instructions': 'Attach a rope handle to a low pulley and stand facing the cable machine. Squat down while holding the rope at chest height.'}
{'id': 192, 'exercise_name': 'Tire Flip', 'type_of_activity': 'Strength', 'type_of_equipment': 'Tire', 'body_part': 'Full Body', 'type': 'Push', 'muscle_groups_activated': 'Glutes, Quadriceps, Hamstrings, Shoulders', 'instructions': 'Squat down and grab the edge of a tire. Lift the tire and 

In [38]:
final_results = []

for doc_id, questions in results.items():
    for q in questions:
        final_results.append((doc_id, q))

In [39]:
final_results[0]

(0, 'What equipment do I need to perform push-ups?')

In [40]:
df_results = pd.DataFrame(final_results, columns=['id', 'question'])

In [41]:
df_results.to_csv('../data/ground-truth-retrieval.csv', index=False)

In [55]:
#!head ../data/ground-truth-retrieval.csv

In [53]:
df_show = pd.read_csv('../data/ground-truth-retrieval.csv')

In [56]:
df_show.head(10)

,id,question
0,0,What equipment do I need to perform push-ups?
1,0,Which muscle groups are primarily worked durin...
2,0,Can you describe the starting position for pus...
3,0,What motion should I follow after lowering my ...
4,0,How should I position my hands when doing push...
5,1,What is the primary muscle group targeted by s...
6,1,Can squats be performed without any equipment?
7,1,How should I position my feet while doing squats?
8,1,What is the proper form to maintain while perf...
9,1,What type of exercise are squats classified as?
